In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-10 01:51:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2022-05-10 01:51:57 (1.21 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("group-2-tornadoes").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# **Load S3 Tornado-Population data into Spark DataFrame**

In [ ]:
# Read in data from S3 buckets
from pyspark import SparkFiles
url = "https://group-2-project-tornadoes.s3.amazonaws.com/joined_torn_pop_data_2010-2019_db.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("joined_torn_pop_data_2010-2019_db.csv"), sep=",", header=True)
df.show()

+-------+---------+----+-----+---+-------------------+-------+---------+---------+---------+-------+--------+---+--------+----------+------+---------+------+-----+--------+
|torn_id|county_id|year|month|day|          timestamp|st_fips|cnty_fips|start_lat|start_lon|end_lat| end_lon| ef|injuries|fatalities|  loss|crop_loss|length|width|cnty_pop|
+-------+---------+----+-----+---+-------------------+-------+---------+---------+---------+-------+--------+---+--------+----------+------+---------+------+-----+--------+
|   6974|    29181|2013|   10| 31|2013-10-31 17:11:00|     29|      181|  36.5862| -90.7416|36.5744|-90.6681|  1|       0|         0|100000|     null|  4.16|  150|   13985|
|   6975|    29023|2013|   10| 31|2013-10-31 17:30:00|     29|       23|  36.6338|   -90.47|36.6351|-90.4664|  1|       0|         0| 80000|     null|  0.22|   50|   42961|
|   6976|    29023|2013|   10| 31|2013-10-31 17:48:00|     29|       23|  36.6954| -90.2691|36.7363|-90.1798|  1|       0|         0|10